In [9]:
import torch
import torch.nn.functional as F

In [10]:
words = open('names.txt', 'r').read().splitlines()

In [11]:
chars = sorted(list(set(''.join(words))))
print(chars)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [12]:
# create a two way dictionray for encoding
stoi= {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(stoi)
print(itos)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [13]:
# create the training set of bigrams (x, y)
xs, ys = [] , []

for w in words:
    chs = ['.']+ list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1= stoi[ch1]
        ix2= stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
        
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('num of examples: ',num )

# randomly initialize 27 neurons' weights, each neuron recives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27),generator=g, requires_grad=True)

num of examples:  228146


In [14]:
# gradient descent
for k in range(100):
    
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float() # inputs to the network: one-hot encoding
    logits = (xenc @ W) # predict log-counts
    counts = logits.exp() # equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # the last two lines are just softmax
    # to add regularization we add some regularization loss
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01 * (W**2).mean()
#     print(loss.item())
    
    # backward pass
    W.grad = None # set gradient to 0
    loss.backward()
    
    # update
    W.data += -50 * W.grad

In [15]:
print("hello world!")

hello world!


In [16]:
# sampling :)

g = torch.Generator().manual_seed(2147483647)
for i in range(5):
    out = []
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float() # inputs to the network: one-hot encoding
        logits = (xenc @ W) # predict log-counts
        counts = logits.exp() # equivalent to N
        p = counts / counts.sum(1, keepdims=True) # probabilities for next character
        # the last two lines are just softmax
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

mor.
axx.
minaymoryles.
kondmaisah.
anchthizarie.
